#### Задача 1 (10 баллов). 

Напишите программу - базу данных для ветеринарной клиники (мы что-то подобное уже делали, если есть старый код - можно его доработать). В базе данных (можете для хранения смело использовать pickle) хранятся записи о питомцах, их хозяевах и даты записи на прием, а также комментарии ветеринара, если прием уже состоялся. Таким образом, нам понадобятся:

- класс "клиент", в котором будет храниться имя и контакты хозяина, а также его питомцы. 
- класс "питомец", в котором будут храниться имя и возраст питомца, даты записи на прием (можно реализовать как меняющийся атрибут "текущий прием", если питомца хотят привести, и атрибут-список или словарь "приемы", куда будут добавляться прошедшие приемы). 
- общий класс "база", где будут храниться клиенты. По базе можно осуществлять поиск клиента по имени или по номеру телефона. 
- дескрипторы / property, чтобы нельзя было вместо имени записать чепуху, указать питомцу возраст недопустимого значения и можно еще телефон проверять. 
- в основном коде программы должна быть возможность посмотреть записи о прошедших приемах, наличие записи на текущий прием; возможность отыскать клиента и установить для одного из его питомцев (видимо, понадобится уметь выводить список питомцев) дату нового приема, а также отметить, что прием уже состоялся, и внести комментарий ветеринара.
- Ну и, конечно, базу данных можно сохранять. Можно это делать при выборе "выйти из программы". 

In [1]:
import re
from datetime import datetime
import pickle


class Human:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.number = ''
        self.pets = []

    def __repr__(self):
        return f"{self.surname} {self.name}, контактный номер: {self.number}"

    def getnumber(self):
        print('getting number')
        return self.number
        
    def setnumber(self, number):
        pattern = r"^(?:\+7|7|8)[-\s]?\d{3}[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}$"
        if bool(re.match(pattern, number)):
            self.number = number
    
    def delnumber(self):
        print('deleting name')
        del self.number
    number = property(getnumber, setnumber, delnumber)

    def add_pets(self, pet):
        if isinstance(pet, Pet):
            self.pets.append(pet)


class Pet:
    def __init__(self, kind, name,  ):
         self.kind = kind
         self.name = name
         self.age = 0
         self.visits = []
         self.current_visit = None

    def __repr__(self):
        return f"{self.kind} {self.name}, возраст {self.age}"

    def getage(self):
        return self.age
        
    def setage(self, age):
        if isinstance(age, int):
            if self.kind != 'черепаха': 
                if 1 <= age <= 20:
                     self.age = age
                self.age = age
        
    def delage(self):
        print('deleting temperature')
        del self.age
    temperature = property(getage, setage, delage)

    def add_visit(self, visit):
        if isinstance(visit, Visit):
            self.current_visit = visit

    def finish_current_visit(self, diagnosis):
        if self.current_visit:
            self.current_visit.diagnosis = diagnosis
            self.visits.append(self.current_visit)
            self.current_visit = None

    def show_visits(self):
        if not self.visits:
            print(f"У {self.name} пока нет приёмов.")
        else:
            print(f"Приёмы {self.name}:")
            for visit in self.visits:
                print(f"Дата: {visit.date}, Статус: {'Прошедший' if visit.status == 'P' else 'Будущий'}, Диагноз: {visit.diagnosis}")
    
class Date:
    """descriptor"""
    def is_date(self, string):
        formats = ['%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y']
        for date_format in formats:
            try:
                datetime.strptime(string, date_format)
                return True
            except ValueError:
                continue
        return False


    def __get__(self, visit, type=None):
        if hasattr(visit, 'date'):
            return visit.date
        raise AttributeError
    
    def __set__(self, visit, value):
        if self.is_date(value):
            visit.date = value
        else:
            print('Can\'t set!')

class Visit:
    date = Date()
    def __init__(self):
        self.diagnosis = ''
        if self.is_date_past():
            self.status = 'P'
        else:
            self.status = 'F'


    def getd(self):
        print('getting diagnosis')
        return self.__diagnosis
        
    def setd(self, diagnosis):
        if "".join(diagnosis.split()).isalpha():
            self.__diagnosis = diagnosis
    
    def deld(self):
        print('deleting diagnosis')
        del self.__diagnosis
    diagnosis = property(getd, setd, deld)



    def is_date_past(self):
        formats = ['%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y']  
        for date_format in formats:
            try:
                input_date = datetime.strptime(self.date, date_format).date() 
                today = datetime.today().date()  
                return input_date < today 
            except ValueError:
                continue  
        return False  


    
class Database:
    def __init__(self):
        self.clients = []

    def add_client(self, client):
        if isinstance(client, Human):
            self.clients.append(client)

    def find_client_by_surname(self, surname):
        for client in self.clients:
            if client.surname == surname:
                return client
        return None

    def find_client_by_pet_name(self, pet_name):
        for client in self.clients:
            for pet in client.pets:
                if pet.name.lower() == pet_name.lower():
                    return client, pet
        return None, None

    def save_database(self, filename="vet_clinic.pkl"):
        with open(filename, 'wb') as file:
            pickle.dump(self.clients, file)

    def load_database(self, filename="vet_clinic.pkl"):
        try:
            with open(filename, 'rb') as file:
                self.clients = pickle.load(file)
        except FileNotFoundError:
            print("Файл базы данных не найден.")


def menu():
    clinic_db = ClinicDatabase()
    clinic_db.load_database()

    while True:
        print("\nМеню:\n1. Добавить клиента\n2. Найти клиента\n3. Назначить прием\n4. Завершить прием\n5. Показать все приемы животного\n6. Выйти и сохранить")
        choice = input("Выберите действие (введите номер)")

        if choice == '1':
            name = input("Введите имя:")
            surname = input("Введите фамилию:")
            number = input("Введите номер телефона:")
            client = Human(name, surname)
            client.number = number
            kind = input("Введите вид животного:")
            pet_name = input("Введите имя животного:")
            new_pet = Pet(kind, pet_name)
            new_pet.age = input("Введите возраст животного:")
            client.add_pets(new_pet)
            clinic_db.add_client(client)
            print(f"Клиент {name} {surname} добавлен.")

        elif choice == '2':
            name = input("Введите фамилию хозяина или имя животного")
            client, pet = clinic_db.find_client_by_pet_name(name)
            if client:
                print(f"Найден клиент: {client.name} {client.surname}, Телефон: {client.number}")
                print(f"Питомец: {pet.name} ({pet.kind}), Возраст: {pet.age}")
            else:
                client = clinic_db.find_client_by_surname(search)
                if client:
                    print(f"Найден клиент: {client.name} {client.surname}, Телефон: {client.number}")
                    print("Питомцы:")
                    for pet in client.pets:
                        print(f"{pet.name} ({pet.kind}), Возраст: {pet.age}")
                else:
                    print("Клиент или питомец не найден.")

        elif choice == '3':  
            search = input("Введите фамилию хозяина")
            client = clinic_db.find_client_by_surname(search)
    
            if client:
              print(f"Клиент найден: {client.name} {client.surname}")
              for i, pet in enumerate(client.pets):
                 print(f"{i + 1}. {pet.name} ({pet.kind}), Возраст: {pet.age}")
        
              pet_index = int(input("Введите номер питомца, для которого хотите назначить прием: ")) - 1
        
              if 0 <= pet_index < len(client.pets):
                  pet = client.pets[pet_index]
                  date = input("Введите дату приема (дд.мм.гггг): ")
                  visit = Visit()
                  visit.date = date  
                  pet.add_visit(visit)  
                  print(f"Прием для {pet.name} назначен на {date}")
              else:
                    print('Питомец не найден. Добавляем новое животное в базу.')
                    kind = input("Введите вид животного:")
                    pet_name = input("Введите имя животного:")
                    new_pet = Pet(kind, pet_name)
                    new_pet.age = input("Введите возраст животного:")
                    client.add_pets(new_pet)
                    date = input("Введите дату приема (дд.мм.гггг): ")
                    visit = Visit()
                    visit.date = date  
                    new_pet.add_visit(visit)  
                    print(f"Создан новый питомец {new_pet.name} и назначен прием на {date}")
            else:
                print("Клиент или питомец не найден.")

        elif choice == '4': 
            search = input("Введите фамилию хозяина или имя животного: ")
            client, pet = clinic_db.find_client_by_pet_name(search)
            
            if client:
                print(f"Клиент найден: {client.name} {client.surname}")
                for index, pet in enumerate(client.pets):
                    print(f"{index + 1}. {pet.name} ({pet.kind}), Возраст: {pet.age}")
    
                pet_index = int(input("Введите номер питомца, для которого хотите завершить прием: ")) - 1
                
                if 0 <= pet_index < len(client.pets):
                    pet = client.pets[pet_index]
                    if pet.current_visit:
                        diagnosis = input("Введите диагноз: ")
                        pet.finish_current_visit(diagnosis)
                        print(f"Прием для {pet.name} завершен.")
                    else:
                        print(f"У {pet.name} нет активного приема.")
                else:
                    print("Некорректный номер питомца.")
            else:
                print("Клиент или питомец не найден.")

        elif choice == '5':  
            search = input("Введите фамилию хозяина или имя животного: ")
            client, pet = clinic_db.find_client_by_pet_name(search)
            
            if client:
                print(f"Клиент найден: {client.name} {client.surname}")
                for index, pet in enumerate(client.pets):
                    print(f"{index + 1}. {pet.name} ({pet.kind}), Возраст: {pet.age}")
                
                pet_index = int(input("Введите номер питомца, для которого хотите посмотреть приемы: ")) - 1
                
                if 0 <= pet_index < len(client.pets):
                    pet = client.pets[pet_index]
                    pet.show_visits() 
                else:
                    print("Некорректный номер питомца.")
            else:
                print("Клиент или питомец не найден.")

        elif choice == '6':
            clinic_db.save_database()
            break




In [ ]:
menu()

Файл базы данных не найден.

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить
Клиент 1 Сергей добавлен.

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить
Клиент или питомец не найден.

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Завершить прием
5. Показать все приемы животного
6. Выйти и сохранить
Клиент или питомец не найден.

Меню:
1. Добавить клиента
2. Найти клиента
3. Назначить прием
4. Заве

    Эту задачу можно написать в тетрадке, если хочется, но, возможно, будет удобнее в py. 